<a href="https://colab.research.google.com/github/PRAVEENKUMARRAJENDRAN/ipl-data-analysis/blob/master/ipl_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [921 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,552 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Pa

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType

In [ ]:
matches_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("season", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("date", DateType(), True),
    StructField("match_type", StringType(), True),
    StructField("player_of_match", StringType(), True),
    StructField("venue", StringType(), True),
    StructField("team1", StringType(), True),
    StructField("team2", StringType(), True),
    StructField("toss_winner", StringType(), True),
    StructField("toss_decision", StringType(), True),
    StructField("winner", StringType(), True),
    StructField("result", StringType(), True),
    StructField("result_margin", IntegerType(), True),
    StructField("target_runs", IntegerType(), True),
    StructField("super_over", StringType(), True),
    StructField("method", StringType(), True),
    StructField("umpire1", StringType(), True),
    StructField("umpire2", StringType(), True)
])

In [ ]:
matches_df = spark.read.schema(matches_schema).format("csv").option("header", "true").load("/content/drive/MyDrive/Spark Data/matches.csv")
matches_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- season: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- date: date (nullable = true)
 |-- match_type: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: integer (nullable = true)
 |-- target_runs: integer (nullable = true)
 |-- super_over: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)



In [ ]:
matches_df.show(10)

+------+------+----------+----------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+----------+------+-------+---------+
|    id|season|      city|      date|match_type|player_of_match|               venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|target_runs|super_over|method|umpire1|  umpire2|
+------+------+----------+----------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+----------+------+-------+---------+
|335982|  null| Bangalore|2008-04-18|    League|    BB McCullum|M Chinnaswamy Sta...|Royal Challengers...|Kolkata Knight Ri...|Royal Challengers...|        field|Kolkata Knight Ri...|   runs|          140|        223|        20|     N|   

In [ ]:
matches_2024_df = matches_df.filter(matches_df.season == 2024)

In [ ]:
matches_2024_df.show(matches_2024_df.count())

+-------+------+-------------+----------+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+----------+------+-------+--------------------+
|     id|season|         city|      date| match_type|    player_of_match|               venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|target_runs|super_over|method|umpire1|             umpire2|
+-------+------+-------------+----------+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+----------+------+-------+--------------------+
|1422119|  2024|      Chennai|2024-03-22|     League|  Mustafizur Rahman|MA Chidambaram St...|Royal Challengers...| Chennai Super Kings|Royal Challengers...|          bat| Chenna

In [ ]:
#highest target rusn to be chased.
import pyspark.sql.functions as F
from pyspark.sql.functions import col


#String or Sql Expression
matches_2024_df.orderBy(expr("target_runs").desc()).select("team1", "team2", "target_runs").show(1)

#Column Expression
matches_2024_df.select("team1", "team2", "target_runs").sort(F.col("target_runs").desc()).show(1)


+-------------------+--------------------+-----------+
|              team1|               team2|target_runs|
+-------------------+--------------------+-----------+
|Sunrisers Hyderabad|Royal Challengers...|        288|
+-------------------+--------------------+-----------+
only showing top 1 row

+-------------------+--------------------+-----------+
|              team1|               team2|target_runs|
+-------------------+--------------------+-----------+
|Sunrisers Hyderabad|Royal Challengers...|        288|
+-------------------+--------------------+-----------+
only showing top 1 row



In [ ]:
matches_2024_df.select("team1", "team2", "target_runs").sort(col("target_runs").asc()).show(1)

matches_2024_df.orderBy(col("target_runs").asc()).select("team1", "team2", "target_runs").show(1)

+--------------+--------------+-----------+
|         team1|         team2|target_runs|
+--------------+--------------+-----------+
|Gujarat Titans|Delhi Capitals|         90|
+--------------+--------------+-----------+
only showing top 1 row

+--------------+--------------+-----------+
|         team1|         team2|target_runs|
+--------------+--------------+-----------+
|Gujarat Titans|Delhi Capitals|         90|
+--------------+--------------+-----------+
only showing top 1 row



In [ ]:
#Highest that is chased Successful
matches_2024_df.select("winner", "target_runs"). \
                filter(col("result") == 'wickets'). \
                sort(col("target_runs").desc()).show(1)

+------------+-----------+
|      winner|target_runs|
+------------+-----------+
|Punjab Kings|        262|
+------------+-----------+
only showing top 1 row



In [ ]:
#Lowest Score that was Defended.
matches_2024_df.select("winner", "target_runs"). \
                filter(col("result") == 'runs'). \
                sort(col("target_runs").asc()).show(1)

+--------------------+-----------+
|              winner|target_runs|
+--------------------+-----------+
|Kolkata Knight Ri...|        158|
+--------------------+-----------+
only showing top 1 row



In [ ]:
#Each Individual team winning when they bat first
matches_2024_df. \
    filter(F.expr("""
    (toss_decision = 'bat' AND toss_winner = winner) OR
    (toss_decision = 'field' AND toss_winner != winner)
    """)).\
    groupBy("winner"). \
    agg(F.count("winner").alias("Number of Wins Bating First")). \
    sort(F.col("Number of Wins Bating First").desc()).\
    show()


+--------------------+---------------------------+
|              winner|Number of Wins Bating First|
+--------------------+---------------------------+
|Kolkata Knight Ri...|                          6|
| Sunrisers Hyderabad|                          6|
|      Delhi Capitals|                          5|
|Lucknow Super Giants|                          4|
|Royal Challengers...|                          4|
| Chennai Super Kings|                          4|
|      Gujarat Titans|                          2|
|      Mumbai Indians|                          2|
|    Rajasthan Royals|                          2|
+--------------------+---------------------------+



In [ ]:
  matches_2024_df.\
    filter(F.expr(
        """
        (toss_decision = 'field' AND winner = toss_winner) OR
        (toss_decision = 'bat' AND winner != toss_winner)
        """
    )). \
    groupBy("winner"). \
    agg(F.count("winner").alias("Number of Wins Fielding First")). \
    sort(F.col("Number of Wins Fielding First").desc()).\
    show()

+--------------------+-----------------------------+
|              winner|Number of Wins Fielding First|
+--------------------+-----------------------------+
|    Rajasthan Royals|                            7|
|Kolkata Knight Ri...|                            5|
|        Punjab Kings|                            5|
|      Gujarat Titans|                            3|
| Sunrisers Hyderabad|                            3|
|Lucknow Super Giants|                            3|
|Royal Challengers...|                            3|
| Chennai Super Kings|                            3|
|      Delhi Capitals|                            2|
|      Mumbai Indians|                            2|
+--------------------+-----------------------------+



In [ ]:
#Each Individual Team winning when they field first
matches_2024_df.select("toss_winner", "toss_decision", "winner"). \
    filter(
        expr("""
            (toss_decision = 'bat' AND toss_winner = winner) OR
            (toss_decision = 'field' AND toss_winner != winner)
        """)
    ).show(matches_2024_df.count())

matches_2024_df.select("toss_winner", "toss_decision", "winner"). \
    filter(
        expr("""
            (toss_decision = 'field' AND toss_winner = winner) OR
            (toss_decision = 'bat' AND toss_winner != winner)
        """)
    ). \
    show(matches_2024_df.count())

+--------------------+-------------+--------------------+
|         toss_winner|toss_decision|              winner|
+--------------------+-------------+--------------------+
| Sunrisers Hyderabad|        field|Kolkata Knight Ri...|
|    Rajasthan Royals|          bat|    Rajasthan Royals|
|      Mumbai Indians|        field|      Gujarat Titans|
|      Gujarat Titans|        field| Chennai Super Kings|
|      Mumbai Indians|        field| Sunrisers Hyderabad|
|      Delhi Capitals|        field|    Rajasthan Royals|
|Lucknow Super Giants|          bat|Lucknow Super Giants|
|      Delhi Capitals|          bat|      Delhi Capitals|
|Royal Challengers...|        field|Lucknow Super Giants|
|Kolkata Knight Ri...|          bat|Kolkata Knight Ri...|
|      Delhi Capitals|        field|      Mumbai Indians|
|Lucknow Super Giants|          bat|Lucknow Super Giants|
|        Punjab Kings|        field| Sunrisers Hyderabad|
|      Mumbai Indians|        field| Chennai Super Kings|
|Royal Challen

In [ ]:
deliveries_df = spark.read.format("csv").option("header", "true").option("inferSchema", True).load("/content/drive/MyDrive/Spark Data/deliveries.csv")
deliveries_df.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- inning: integer (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)
 |-- over: integer (nullable = true)
 |-- ball: integer (nullable = true)
 |-- batter: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- batsman_runs: integer (nullable = true)
 |-- extra_runs: integer (nullable = true)
 |-- total_runs: integer (nullable = true)
 |-- extras_type: string (nullable = true)
 |-- is_wicket: integer (nullable = true)
 |-- player_dismissed: string (nullable = true)
 |-- dismissal_kind: string (nullable = true)
 |-- fielder: string (nullable = true)



In [ ]:
deliveries_df.show()

+--------+------+--------------------+--------------------+----+----+-----------+---------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|match_id|inning|        batting_team|        bowling_team|over|ball|     batter|   bowler|non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|
+--------+------+--------------------+--------------------+----+----+-----------+---------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   1| SC Ganguly|  P Kumar|BB McCullum|           0|         1|         1|    legbyes|        0|              NA|            NA|     NA|
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   2|BB McCullum|  P Kumar| SC Ganguly|           0|         0|         0|       null|        0|              NA|            NA|    

In [ ]:
deliveries_2024_df = deliveries_df.join(matches_2024_df, deliveries_df.match_id == matches_2024_df.id)
deliveries_2024_df.show()

+--------+------+--------------------+-------------------+----+----+------------+------------+------------+------------+----------+----------+-----------+---------+----------------+--------------+-------+-------+------+-------+----------+----------+-----------------+--------------------+--------------------+-------------------+--------------------+-------------+-------------------+-------+-------------+-----------+----------+------+-------+----------+
|match_id|inning|        batting_team|       bowling_team|over|ball|      batter|      bowler| non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|     id|season|   city|      date|match_type|  player_of_match|               venue|               team1|              team2|         toss_winner|toss_decision|             winner| result|result_margin|target_runs|super_over|method|umpire1|   umpire2|
+--------+------+--------------------+-------------------+----+----+------------+-------

In [ ]:
deliveries_2024_df. \
    groupBy("batter", "batting_team"). \
    agg(F.sum("batsman_runs").alias("Total_Runs")). \
    sort(F.col("Total_Runs").desc()). \
    show(1)

+-------+--------------------+----------+
| batter|        batting_team|Total_Runs|
+-------+--------------------+----------+
|V Kohli|Royal Challengers...|       741|
+-------+--------------------+----------+
only showing top 1 row



In [ ]:
deliveries_2024_df. \
    filter(F.expr("dismissal_kind != 'run out'")). \
    groupBy("bowler", "bowling_team"). \
    agg(F.sum("is_wicket").alias("Total_Wickets")). \
    sort(F.col("Total_Wickets").desc()). \
    show(1)

+--------+------------+-------------+
|  bowler|bowling_team|Total_Wickets|
+--------+------------+-------------+
|HV Patel|Punjab Kings|           24|
+--------+------------+-------------+
only showing top 1 row



In [ ]:
deliveries_2024_df. \
    filter(F.expr("extras_type = 'wides'")). \
    groupBy("bowler", "bowling_team"). \
    agg(F.count("extras_type").alias("Total_Extras")). \
    sort(F.col("Total_Extras").desc()). \
    show()

+-----------------+--------------------+------------+
|           bowler|        bowling_team|Total_Extras|
+-----------------+--------------------+------------+
|   Mohammed Siraj|Royal Challengers...|          26|
|   Arshdeep Singh|        Punjab Kings|          24|
|      T Natarajan| Sunrisers Hyderabad|          24|
|         KK Ahmed|      Delhi Capitals|          23|
|         HV Patel|        Punjab Kings|          21|
|     TU Deshpande| Chennai Super Kings|          20|
|         I Sharma|      Delhi Capitals|          19|
|       Yash Dayal|Royal Challengers...|          18|
|         R Ashwin|    Rajasthan Royals|          16|
|       Avesh Khan|    Rajasthan Royals|          16|
|         MA Starc|Kolkata Knight Ri...|          16|
|         VG Arora|Kolkata Knight Ri...|          15|
|Mustafizur Rahman| Chennai Super Kings|          15|
|       N Thushara|      Mumbai Indians|          14|
|        G Coetzee|      Mumbai Indians|          14|
|          B Kumar| Sunriser

In [ ]:
deliveries_2024_df. \
    select("bowling_team", "bowler", "extras_type"). \
    filter(F.expr("extras_type = 'wides'")). \
    sort(F.col("bowling_team").asc(), F.col("bowler").asc()). \
    show(5000)

+--------------------+-------------------+-----------+
|        bowling_team|             bowler|extras_type|
+--------------------+-------------------+-----------+
| Chennai Super Kings|        DJ Mitchell|      wides|
| Chennai Super Kings|        DJ Mitchell|      wides|
| Chennai Super Kings|          DL Chahar|      wides|
| Chennai Super Kings|          DL Chahar|      wides|
| Chennai Super Kings|          DL Chahar|      wides|
| Chennai Super Kings|          DL Chahar|      wides|
| Chennai Super Kings|          DL Chahar|      wides|
| Chennai Super Kings|        M Pathirana|      wides|
| Chennai Super Kings|        M Pathirana|      wides|
| Chennai Super Kings|        M Pathirana|      wides|
| Chennai Super Kings|        M Pathirana|      wides|
| Chennai Super Kings|        M Pathirana|      wides|
| Chennai Super Kings|        M Pathirana|      wides|
| Chennai Super Kings|        M Pathirana|      wides|
| Chennai Super Kings|        M Pathirana|      wides|
| Chennai 

In [ ]:
deliveries_2024_df. \
    select("batter", "batting_team", "batsman_runs"). \
    groupBy("batter", "batting_team"). \
    agg(F.sum("batsman_runs").alias("total_runs")). \
    sort(F.col("batting_Team").asc(), F.col("Total_runs").desc()). \
    show(5000)

+-------------------+--------------------+----------+
|             batter|        batting_team|total_runs|
+-------------------+--------------------+----------+
|         RD Gaikwad| Chennai Super Kings|       583|
|             S Dube| Chennai Super Kings|       396|
|        DJ Mitchell| Chennai Super Kings|       318|
|          RA Jadeja| Chennai Super Kings|       267|
|          AM Rahane| Chennai Super Kings|       242|
|         R Ravindra| Chennai Super Kings|       222|
|           MS Dhoni| Chennai Super Kings|       161|
|             MM Ali| Chennai Super Kings|       128|
|       Sameer Rizvi| Chennai Super Kings|        51|
|          SN Thakur| Chennai Super Kings|        21|
|         MJ Santner| Chennai Super Kings|        14|
|         RJ Gleeson| Chennai Super Kings|         2|
|       TU Deshpande| Chennai Super Kings|         0|
|            RR Pant|      Delhi Capitals|       446|
|           T Stubbs|      Delhi Capitals|       378|
|    J Fraser-McGurk|      D